## Load the Motion Data

Data is in x and y coordinates for each pixel. Each sample will be an array of 10 (frames) x 40 x 40 (capture window) x 2 (x and y) dimensions. 

In [ ]:
import numpy as np
import os
import matplotlib
matplotlib.use('TkAgg') # For displaying animation
import matplotlib.pyplot as plt
import matplotlib.animation as animation

from helper_functions import *
from normalize_data import *
from integral_try import *
from numpy import array
from sklearn import preprocessing as pp
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [ ]:
data_dir = 'sonic_pi_face/data/'

# Get list of data files
data_files = get_data_files(data_dir)

# Load data into a dictionary
# Note: Checks for incomplete data
data = get_gesture_data(data_files)

# Normalize data across data samples and frames (x and y will be normalized independently)
# data = normalize_data(data)
print(np.shape(data['open-close']))

# Load first data sample from `open-close`
sample = data['open-close'][0]
print(np.shape(sample))

# calculate the integral image
iimage = i_image(sample[4])

# calculate the integral for the first quarter from the integral image
try_integral = get_integral(iimage,0,0,20,20)

print('Example integral of the 1st quarter of one frame: ' , try_integral)

## Visualize horizontal motion across frames

In [ ]:
sample = data['open-close'][121] # Fifth sample
frame = sample[5][:,:,0] # Middle frame
# Note: Second argument takes 0 for `x`, 1 for `y` coordinate
# display_frames(sample,0) 

## Random forest classification

Create training and test data split for classification

In [ ]:
# Load gesture data
open_close = data['open-close']
oc_array = np.asarray(open_close)

# Flatten array to n x 32000
flattened = oc_array.reshape((len(oc_array),10*1600*2))

In [ ]:
# Load empty (no-gesture) data
emptyX = np.asarray(data['empty']).reshape((len(emptyX),10*1600*2))
emptyY = np.zeros(len(emptyX))

In [ ]:
dataX = flattened.copy()

# create random mock data on same scale as original data
shamX = np.random.randn(*dataX.shape) * (np.max(dataX) + np.abs(np.min(dataX))) - np.min(dataX)
X = np.vstack([dataX, shamX,emptyX])
dataY = np.ones((len(dataX)))
shamY = np.zeros((len(shamX)))
Y = np.hstack([dataY, shamY, emptyY])

In [ ]:
# Split data into train and test
X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state=42)

# Classify test data using random forest
clf = RandomForestClassifier(n_estimators=10)
clf = clf.fit(X_train, y_train)
clf.score(X_test,y_test)